In [5]:
import numpy as np
import sys
sys.path.append(r"C:\Users\gapar\Logical-Clifford-Synthesis")
from Algorithms.algorithm_1 import SymplecticOperations
from helper_functions.helperfunctions import SymplecticMatrix
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# Initialize the SymplecticOperations class
symplectic_ops = SymplecticOperations()
symp_mat = SymplecticMatrix(3)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import numpy as np

def reduce_to_echelon_form(A, b):
    """Reduce the matrix A to echelon form over GF(2) and apply the same operations to b."""
    rows, cols = A.shape
    A = np.copy(A) % 2
    b = np.copy(b) % 2
    row = 0

    for col in range(cols):
        if row >= rows:
            break
        
        # Find a pivot row for this column
        pivot = np.where(A[row:, col] == 1)[0]
        if len(pivot) == 0:
            continue  # No pivot in this column, move to the next column
        pivot_row = pivot[0] + row
        
        # Swap the current row with the pivot row
        A[[row, pivot_row]] = A[[pivot_row, row]]
        b[[row, pivot_row]] = b[[pivot_row, row]]
        
        # Eliminate all other ones in this column
        for r in range(rows):
            if r != row and A[r, col] == 1:
                A[r] ^= A[row]
                b[r] ^= b[row]
        row += 1

    return A, b

def solve_from_echelon_form(A, b):
    """Solve the system given A in echelon form."""
    rows, cols = A.shape
    x = np.zeros(cols, dtype=int)
    
    for r in range(rows - 1, -1, -1):
        if np.any(A[r]) == 0:
            continue  # Skip rows that are all zeros
        col = np.argmax(A[r])  # Find the first 1 in this row
        x[col] = b[r] ^ np.dot(A[r], x) % 2
    return x

# Example
A = np.array([[1,0,1,0], [1,0,0,0],[0,1,0,1]], dtype=int)
b = np.array([1,1,1], dtype=int)

A_reduced, b_reduced = reduce_to_echelon_form(A, b)
x = solve_from_echelon_form(A_reduced, b_reduced)

print("Reduced A:\n", A_reduced)
print("Modified b:", b_reduced)
print("Solution:", x)


Reduced A:
 [[1 0 0 0]
 [0 1 0 1]
 [0 0 1 0]]
Modified b: [1 1 0]
Solution: [1 1 0 0]


In [9]:
def solve_gf2(A, B):
    """
    Solve the system of equations Ax = B in GF(2) using the inverse method.
    
    Parameters:
    - A: Coefficient matrix in GF(2)
    - B: Constant vector in GF(2)
    
    Returns:
    - x: Solution vector in GF(2)
    
    Raises:
    - ValueError: If A is singular (non-invertible) in GF(2).
    """
    # Step 1: Compute the inverse of A in GF(2)
    try:
        A_inv = symp_mat.gf2matinv(A)
        print(A_inv)
    except ValueError as e:
        return str(e)  # If A is singular, return the error message
    
    # Step 2: Multiply A_inv by B to find x
    x = np.dot(A_inv, np.transpose(B)) % 2  # Perform dot product and reduce mod 2 (GF(2) arithmetic)
    
    return x


In [10]:
solution = solve_gf2(A, b)
print("Solution:", x)

[[0 0 1 0]
 [1 0 0 1]
 [0 1 1 0]]


ValueError: shapes (3,4) and (3,) not aligned: 4 (dim 1) != 3 (dim 0)